# Build a Dashboard Application with Plotly Dash

In this lab, we will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. We will be guided to build this dashboard application via the following tasks:

* TASK 1: Add a Launch Site Drop-down Input Component
* TASK 2: Add a callback function to render **success-pie-chart** based on selected site dropdown
* TASK 3: Add a Range Slider to Select Payload
* TASK 4: Add a callback function to render the **success-payload-scatter-chart** scatter plot

After visual analysis using the dashboard, we should be able to obtain some insights to answer the following five questions:

1. Which site has the largest successful launches?
1. Which site has the highest launch success rate?
1. Which payload range(s) has the highest launch success rate?
1. Which payload range(s) has the lowest launch success rate?
1. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?

Estimated time needed: **90 minutes**



In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import wget

 Download a skeleton dashboard application and dataset

In [2]:
# Uncomment the following cell to get download skeleton app and dataset:

# spacex_dataset = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
# spacex_dash_app = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py")

Complete tasks in the dashboard:

In [5]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)
server = app.server

# Create an app layout

app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                html.P("Created by Molo Munyansanga", style={'textAlign': 'center'}),
                                
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                                options=[
                                                    {'label': 'All Sites', 'value': 'ALL'},
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                ],
                                                value='ALL',
                                                placeholder="Select a Launch Site here",
                                                searchable=True
                                                ),

                                html.Br(),

                                # Task 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected,
                                # show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                # Add payload mass slider text
                                html.P(id="slider-text"),
                                
                                # TASK 3: Add a slider to select payload range
                                # dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500',
                                                       10000: '10000'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])


# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Successful Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        site_chosen = entered_site
        mask = filtered_df['Launch Site'] == site_chosen
        fig = px.pie(filtered_df[mask], names='class',
                     title=f'Total Successful Launches For Site {site_chosen}')
        return fig


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              [Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site, mass):

    # filter masses from payload slider
    mass_1 = spacex_df['Payload Mass (kg)'] >= float(mass[0])
    mass_2 = spacex_df['Payload Mass (kg)'] <= float(mass[1])
    
    filtered_df = spacex_df[mass_1][mass_2]
    
    if entered_site == 'ALL':

        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color="Booster Version Category",
                         title=f'Correlation between Payload Mass and Launch Success for All Sites for Payload Mass(kg) Between {mass[0]} and {mass[1]}')
        return fig
    else:
        
        # return the outcomes scatter chart for a selected site
        site_chosen = entered_site
        mask = filtered_df['Launch Site'] == site_chosen
        fig = px.scatter(filtered_df[mask], x='Payload Mass (kg)', y='class', color="Booster Version Category",
                         title=f'Correlation between Payload Mass and Launch Success for Site {site_chosen}')
        return fig
    
    
#function to return payload_mass success_rate
@app.callback(Output('slider-text', 'children'),
              [Input(component_id='payload-slider', component_property='value')])
def get_success_rate(mass):

    # filter masses from payload slider
    mass_1 = spacex_df['Payload Mass (kg)'] >= float(mass[0])
    mass_2 = spacex_df['Payload Mass (kg)'] <= float(mass[1])
    
    filtered_df = spacex_df[mass_1][mass_2]
    
    # find success rate
    rate = (filtered_df['class'].value_counts().loc[1])/filtered_df['class'].value_counts().sum() * 100
    rate = 'Payload range (Kg): ' + str(round(rate, 2)) + '% Success Rate'
    
    return rate
    
    
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Mar/2022 16:47:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2022 16:47:31] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2022 16:47:31] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2022 16:47:32] "GET /_favicon.ico?v=2.3.0 HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2022 16:47:32] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2022 16:47:32] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2022 16:47:32] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2022 16:47:32] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2022 16:47:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2022 16:47:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2022 16:47:32] "POST /_dash-upd

### Dashboard Deployed:

You can view the dashboard on the following link: [spacex-dashboard](https://spacex-capstone-dashboard.herokuapp.com/)

##### Observations:

* Site **KSC LC-39A** has the largest successful launches as well the highest launch success rate.
* It appears that the payload range between 2000 kg and 4000 kg has the highest success rate.
* It also seems like the launch success rate is dramatically low between the payload range of 0kg and 2500kg. Perhaps very low masses decrease launch success.
* The booster version **FT**, seems to have a higher success rate than other booster versions

## Authors


<a href="https://www.linkedin.com/in/molomunyansanga/">Molo Munyansanga</a> is a Data Science enthusiast with certificates in Statistics, Databases and Machine Learning. He is also enrolled in the Data Science Professional Certification program by IBM.

## Change Log


| Date (YYYY-MM-DD) | Version | Changed By    | Change Description      |
| ----------------- | ------- | ------------- | ----------------------- |
| 2022-03-19        | ------  | Molo. M       | Created Notebook and Completed Tasks      |